In [131]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
from time import time, sleep
from datetime import datetime
import random
from json import loads
import json
import re

In [413]:
link = 'https://www.eobuwie.com.pl/oferta'

In [414]:
page = requests.get(link)

In [415]:
soup = BeautifulSoup(page.content)

In [84]:
link_article = soup.find(attrs =  {"class": "products-list__link"})['href'][2:]


In [ ]:
for i in soup.find_all(attrs =  {"class": "products-list__link"}):
  try:
    i['href'][2:]
  except:
    continue
  

In [467]:
soup.find_all(attrs =  {"class": "products-list__link"})[11]['href'][2:]


'www.eobuwie.com.pl/sneakersy-skechers-subtle-spots-155616-bbk-black.html'

In [220]:
# testowo dowolny inny artykul

link_article = "www.eobuwie.com.pl/sneakersy-puma-rose-raw-metallics-383849-03-pristine-rose-gold.html"

In [221]:
page_article = requests.get("http://" + link_article)

In [222]:
soup_article = BeautifulSoup(page_article.content)

In [239]:
try:
   RRP = soup_article.find(attrs = {"class" : "e-product-price__group"}).find(attrs = {"class" : "e-product-price__regular"}).text.strip().replace('\xa0zł',"")
except:
  RRP = soup_article.find(attrs = {"class" : "e-product-price__normal"}).text.strip().replace('\xa0zł',"")
#  else:
 #  soup_article.find(attrs = {"class" : "e-product-price__normal"}).text.strip().replace('\xa0zł',"")


In [240]:
soup_article.find(attrs = {"class" : "e-product-price__group"})

In [242]:
try:
  PRICE = soup_article.find(attrs = {"class" : "e-product-price__group"}).find(attrs = {"class" : "e-product-price__special"}).text.strip().replace('\xa0zł',"")
except:
  PRICE = RRP
else:
  PRICE = RRP

In [244]:
NAZWA = soup_article.find("meta" ,property="product:mfr_part_no")["content"]

In [245]:
RRP, PRICE, NAZWA

('269,99', '269,99', 'Rose Raw Metallics 383849 03')

In [ ]:
###### automated code

In [382]:
# get free IP's from Poland
ip_link = 'http://free-proxy.cz/en/proxylist/country/PL/all/ping/all'
ip_page = requests.get(ip_link)
soup_ip = BeautifulSoup(ip_page.content)


In [383]:
# get table from website
data = []

table = soup_ip.find('table',{'id': 'proxy_list'})
rows = table.find_all('tr')

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

In [385]:
# get only ip number in a list
ip_list = []

# ip on this side are decoded, so we have encode them
for i in data:
  try:
    decoded = i[0].replace("document.write(Base64.decode(","").replace("))","")  
    encoded = base64.b64decode(decoded).decode()
    ip_list.append(encoded)
  except:
    continue

In [400]:
# list of free proxies

proxies = []
for ip in ip_list:
  x = {"http": ip}
  proxies.append(x)

{'http': '216.39.173.51'}

In [402]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

list

In [ ]:
#def scrap_eobuwie(main_url = "https://www.eobuwie.com.pl/producenci:",subpages = ["adidas_performance","puma", "reebok","4f","kappa","new_balance","skechers","under_armour","vans","lee_cooper","ipanema","campus"],  prefix = '.html']

## dodaj iteracje przez wszystkie podstrony

def scrap_eobuie(brands):

  start = time()

    session = requests.Session()
    session.max_redirects = 60
    cookies = dict(cookies_are='working')
    
    # empty dataframe to collect data
    pricelist = pd.DataFrame({"Article" : [],
                 "Price" : [],
                 "RRP": []})
    
    # iterate through each provided brand
    for brand in brands:
      url = "https://www.eobuwie.com.pl/producenci:" + brand + ".html"

      # get whole structure of data, and avoid getting blocked, by add headers parameter
      page = session.get(url, headers = headers,cookies=cookies) 

      # let parse our web page
      soup = BeautifulSoup(page.content),

      # iterate throug all products on one page to get direct link to them
      for product in soup.find_all(attrs =  {"class": "products-list__link"}):
        try:
          link = product['href'][2:]
        except:  ### czy to jest dobre miejsca)
          continue
      
        # choose random proxy
        proxy = random.choice(proxies)

        page_article = session.get(link, headers = headers,proxies = proxy, timeout = 98,cookies=cookies) 
        soup_article = BeautifulSoup(page_article.content)

        # getting information about articels
        try:
          RRP = soup_article.find(attrs = {"class" : "e-product-price__group"}).find(attrs = {"class" : "e-product-price__regular"}).text.strip().replace('\xa0zł',"")
        except:
          RRP = soup_article.find(attrs = {"class" : "e-product-price__normal"}).text.strip().replace('\xa0zł',"")

        try:
          PRICE = soup_article.find(attrs = {"class" : "e-product-price__group"}).find(attrs = {"class" : "e-product-price__special"}).text.strip().replace('\xa0zł',"")
        except:
          PRICE = RRP

        NAME = soup_article.find("meta" ,property="product:mfr_part_no")["content"]


        pricelist = pricelist.append({'Article' : NAME,
                               'Price' : PRICE,
                               'RRP' : RRP},
                              ignore_index = True)
        
        # sleep for one seconds after each link to avoid block
        sleep(1)

    end = time()
    print(round(end - start,2), 'calculation time in sec')
    
    return pricelist


In [411]:
brands = ["adidas_performance","puma", "reebok","4f","kappa","new_balance","skechers","under_armour","vans","lee_cooper","ipanema","campus"]